# Preprocessing text-mining
Ce notebook explore les données ocr burtes afin d'en comprendre les principales caractéristiques. Il ne contient, en lui-meme, pas de résultat mais a permis d'identifier les pistes de traitements qui ont permis:
- de choisir d'utiliser les données ltdrwocr
- d'identifier les améliorations qui seront mises en oevre dans le notebook de preprocessing du texte (4.3)

In [ ]:
import sys
from pathlib import Path

project_root = Path().resolve().parent
if not project_root in [Path(p).resolve() for p in sys.path]:
    sys.path.append(str(project_root))

from src import PATHS

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

# 1. Chargement des données

In [ ]:
df = pd.read_parquet(PATHS.processed_data / "df_raw_ocr.parquet")\
    .join(pd.read_parquet(PATHS.metadata / "df_encoded_labels.parquet"))\
    .join(pd.read_parquet(PATHS.metadata / "data_sets.parquet"))
df.head()

In [ ]:
df = df.dropna()
# df.drop(columns="raw_ocr").groupby("data_set").value_counts()
plt.figure(figsize =(10, 5))
plt.subplot(121)
sns.countplot(
    data = df,
    x = "label",
    hue = "data_set",
    stat="percent"

)
plt.title("En pourcentage du nombre de lignes total")
plt.subplot(122)

# je n'ai pas réussi à faire ce graphe avec seaborn nativement...
df_grouped = (
    df.groupby(["data_set", "label"])
    .size()
    .reset_index(name="count")
)
df_grouped["percent"] = df_grouped.groupby("data_set")["count"].transform(lambda x: x / x.sum() * 100)

sns.barplot(
    data=df_grouped,
    x="label",
    y="percent",
    hue="data_set"
)
plt.title("En pourcentage du nombre de lignes du data_set")
plt.suptitle("Distributions des différents labels")

Les distributions sont globalement uniformes, avec un affaiblissement des catégories 8, 4 et 3.

In [ ]:
train = df[df.data_set == "train"].drop(columns="data_set")
test = df[df.data_set == "test"].drop(columns="data_set")
val = df[df.data_set == "val"].drop(columns="data_set")

A partir de maintenant, nous ne travaillons que sur train

# 2. Etude manuelle des défauts d'OCR

In [ ]:
# OPERATION PREALABLE // A RETIRER QUAND PROCESSUS STABLE

train = train[:1000]

In [ ]:
# Pour faciliter les premières analyses, on va se limiter aux ocr dont le nombre de caractères n'est pas trop long:
train_short_ocrs = train[train.raw_ocr.str.len()<500]
# len(train_short_ocrs) @=197

In [ ]:
# La fonction ocr_insight permet d'afficher une image et son texte océrisé côte à côte.
# Cela va nous permettre de mieux comprendre certains ocr
from PIL import Image
import io
import base64
from IPython.display import display, HTML

image_paths = pd.read_parquet(documents_file)\
    .set_index("document_id")\
    [["rvl_image_path", "iit_image_path"]]

def display_tiff_pages(tiff_path):
    img = Image.open(tiff_path)

    for i in range(img.n_frames):
        img.seek(i)  # Aller à la page i
        buffered = BytesIO()
        img.save(buffered, format="PNG")
        img_b64 = base64.b64encode(buffered.getvalue()).decode()
        
        html_blocks.append(f'<img src="data:image/png;base64,{img_b64}" style="width: 60%; margin-bottom: 20px;" />')

    display(HTML("<br>".join(html_blocks)))




def get_image(document_id):
    """Return a BytesIO png converted image, which will be compatible with html display"""
    image_path = os.path.join(
        data_path,
        image_paths.loc[document_id, "iit_image_path"]
    )
    
    html_blocks = []
    img = Image.open(image_path)
    
    for i in range(img.n_frames):
        img.seek(i)  # Aller à la page i
        buffered = io.BytesIO()
        img.save(buffered, format="PNG")
        img_b64 = base64.b64encode(buffered.getvalue()).decode()
        
        html_blocks.append(f'<img src="data:image/png;base64,{img_b64}" />')
    return html_blocks
    
def ocr_insight(document_id):
    text = train.loc[document_id, "raw_ocr"]
    raw_text = repr(text)
    interpretated_text = '\n'.join([
        f"<p>{line}</p>"
        for line in text.splitlines()])
    image = get_image(document_id)
    
    html_code = f"""
    <div style="display: flex; align-items: center;">
      <div style="width: 40%; max-height: 600px; overflow-y: auto">
        <p><strong>Texte brut :</strong></p>
        <p>{raw_text}</p>
        <p><strong>Texte interprété :</strong></p>
        {interpretated_text}
      </div>
      <div style="width: 60%"; max-height: 600px>
        {"<br>".join(get_image(document_id))}
    </div>@
    </div>
    """
#      <img src="data:image/png;base64,{image}" style="width: 60%; margin-right: 20px;">
    display(HTML(html_code))

In [ ]:
ocr_insight(train_short_ocrs.index[0])


<span style="color:blue">**Analyse :**</span>
- pgNbr inutile
- une typo (6ring qui est sans doute bring)

In [ ]:
ocr_insight(train_short_ocrs.index[1])


<span style="color:blue">**Analyse :**</span>
- OCR inexploitable (aucune information captée du fait de l'écriture manuscrite)
- nombre de pages (2)  difficile à interpréter (le document ne semble en contenir qu'une)
- le nombre de page est un artéfact de la numérisation ==> a supprimer car ce n'est pas une information provenant du document

In [ ]:
ocr_insight(train_short_ocrs.index[2])


<span style="color:blue">**Analyse :**</span>
- OCR parfait (hors pgNbr)

In [ ]:
ocr_insight(train_short_ocrs.index[3])


<span style="color:blue">**Analyse :**</span>
- OCR globalement bon (perturbation sur les taches et manques de précision: "Rw" pour "Rev",...
- texte manuscrit ignorées
- ordre de lecture naturel non respecté (pour le tableau approvals, on parcourt la premiere colonne jusq'a 3 puis les entetes de colonne, puis la suite de la premiere colonne
- artéfact "f li"
- toujours le pgNbr

In [ ]:
ocr_insight(train_short_ocrs.index[4])


<span style="color:blue">**Analyse :**</span>
- non détection du texte blanc sur fond noir
- non détection du texte dans l'encart en bas à gauche (comme si un traitement par lignesavait choisi de ne pas le traiter)
- assez peu de mauvaises détections
- textes du bas souffrant d'erreurs: nicmine pour nicotine, mdds pour milds, rigarette pour cigarette
- toujours le pgNbr

In [ ]:
# @Alexis, je te laisse continuer si tu veux...

## Conclusion
Pour les traitements à venir, nous avons observé qu'il pourrait être utile:
- retirer toutes les informations relatives au pgNbr=x;
- "déséchapper" tous les caractères html (&lt, &amp, ...) // non vu ici mais est facilement identifié avec train[train.raw_ocr.str.contains("&lt")]
- corriger avec un outil spécialisé les erreurs d'océrisation, avec si possible prise en compte du contexte;
- ne pas espérer avoir de résultats sur les textes écrits à la main
- de supprimer les caractères spéciaux, voire les lignes contenant plus de ces caractères erronés que de caractères alphanumériques;
- de conserver aussi longtemps que possible les lignes, qui sont en général cohérentes;
- de considérer, in fine comme na les documents qui ne contiennent plus assez d'information (nombre de mots trop faible)

# 3. Vérification d'hypothèses

## 3.1. Isolement des "Handwritten" par un pattern spécifique
On a remarqué que les ocr correspondant à des handwritten étaient souvent formés de la même manière. Nous allons essayer de trouver un pattern qui permette de les isoler avec un bon score d'accuracy.
Nous travaillons sur le jeu de données train, pour éviter toute fuite de données.

In [ ]:
df_hw_train = df[(df.label == 3) & (df.data_set == "train")]
df_others_train = df[(df.label != 3) & (df.data_set == "train")]

In [ ]:
df_hw_train.head(10)

In [ ]:
# On va verifier combien de documents contiennent plus de 5 mots de deux lettres.


In [ ]:
import re

# A améliorer pour prendre en compte les lignes
# sans doute trop brutal / il faudra le réviser sur d'autres versions ultérieures?
def basic_word_filter(text):
    if not text:
        return text
    text = text.lower()
    # Attention, c'est brutal, ca supprime tous les chiffres aussi...
    word_regex = re.compile(r'[a-z]{4,}')
    text = ' '.join(word_regex.findall(text))
    
    return text

In [ ]:
def word_count(text):
    words = text.split(' ')
    return len(words)
    
text = "pgnbr vt rr zlle am ln oajl amp pgnbr"

In [ ]:
def handwritten_filter(text):
    """says if a text is handwritten or not"""
    if word_count(basic_word_filter(text)) < 15:
        return True
    else:
        return False

In [ ]:
X_train = df[df.data_set == "train"].drop(columns=["label", "data_set"])
y_train = df[df["data_set"] == "train"]["label"] == 3

In [ ]:
y_pred = X_train.raw_ocr.apply(handwritten_filter)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_train, y_pred))

In [ ]:
wc_results = X_train.raw_ocr.apply(basic_word_filter).apply(word_count)

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

plt.figure(figsize=(20,10))
sns.histplot(wc_results.values, bins = [0,1,2,3,4,5, 10, 15, 20, 30, 40, 50,100]);
# plt.xlim(0, 10);

In [ ]:
df_hw_train[df_hw_train.raw_ocr.apply(basic_word_filter).apply(word_count) <10]

In [ ]:
df_others_train[df_others_train.raw_ocr.apply(basic_word_filter).apply(word_count) <10]

In [ ]:
len(df_others_train)

In [ ]:
# Avec un filtre à 10 mots, on arrive a identifier environ 15500 handwritten sur 20000
# dans le même temps, on identifierait à tort 16665 documents sur 290000 
